In [1]:
import praw
import numpy
%matplotlib inline
from pylab import *
from pprint import pprint
import pandas as pd
import simplejson as json
from psaw import PushshiftAPI
import pyspark
import datetime as dt

%config InlineBackend.figure_format='retina'

In [44]:
CLIENT_ID = '5FpjYNUrv93rDqflbNE-6w'
CLIENT_SECRET = '5ZITWnmNd6_qCbPXDB90xmOaIMo9_w'
USER_NAME = 'theonejay96'
SCRIPT_NAME = 'Data\ Science\ Thesis'
LOGIN_PASSWORD = 'vejkyz-hyBfez-4cinte'

reddit =  praw.Reddit(
    client_id= CLIENT_ID ,
    client_secret= CLIENT_SECRET ,
    user_agent= SCRIPT_NAME,
    username= USER_NAME,
    password= LOGIN_PASSWORD)

api = PushshiftAPI()

Version 7.5.0 of praw is outdated. Version 7.6.0 was released Tuesday May 10, 2022.


In [2]:
def importance(df): #calculate the importance (similar to tf-idf)
    # OPEN
    # - make work with Crosspost from
    imp = df.groupby(['crosspost_from']).agg({'subreddit': 'count', 'count': 'sum', 'crosspost_from_num' : 'sum'})
    imp = imp.rename(columns ={'count': 'total'}) 
    imp.drop(['subreddit'], axis=1, inplace=True)
    df2 = df.join(imp, on='crosspost_from', how='left')
    return df2

In [75]:
df2 = pd.read_pickle('../../Files/2021-01-02.pickle')

In [64]:
for i, r in df2.iterrows(): # Pull crosspost_from information from field 'crosspost_parent_list' (which is in a json format)
    # Using the field has the advantage that we can deal with deleted posts which we could not find using the reddit API.
    t = r['crosspost_parent_list'][0]
    r['crosspost_from'] = t['subreddit']
    r['crosspost_from_id'] = t['subreddit_id']
    r['crosspost_from_subs'] = t['subreddit_subscribers'] 
    r['crosspost_from_num'] = t['num_crossposts'] 


IndexError: list index out of range

In [68]:
df3 = df2[df2['crosspost_parent_list'].str.len() == 0]

In [76]:
df2.head()

,index,subreddit,subreddit_id,subreddit_subscribers,crosspost_parent,crosspost_parent_list,created_utc,author,created,t,crosspost_from,crosspost_from_id,crosspost_from_subs,crosspost_from_num
0,0,DebateVaccines,t5_3jwf7,2347,t3_l9iiqp,"[{'all_awardings': [], 'allow_live_comments': ...",1612117931,[deleted],1.612111e+09,"{'all_awardings': [], 'allow_live_comments': F...",conspiracy,t5_2qh4r,1455036,1
1,3,DebateVaccines,t5_3jwf7,2325,t3_l81oft,[{'all_awardings': [{'award_sub_type': 'GLOBAL...,1612056522,doingbasiclifeprep,1.612049e+09,{'all_awardings': [{'award_sub_type': 'GLOBAL'...,IAmA,t5_2qzb6,20780922,1
2,15,DebateVaccines,t5_3jwf7,2317,t3_l7apag,[{'all_awardings': [{'award_sub_type': 'GLOBAL...,1611915334,EuCleo,1.611908e+09,{'all_awardings': [{'award_sub_type': 'GLOBAL'...,Coronavirus,t5_2x4yx,2363477,1
3,17,DebateVaccines,t5_3jwf7,2314,t3_l77ln3,"[{'all_awardings': [], 'allow_live_comments': ...",1611865305,dannylenwinn,1.611858e+09,"{'all_awardings': [], 'allow_live_comments': F...",ForUnitedStates,t5_2i3wxn,2757,26
4,18,DebateVaccines,t5_3jwf7,2314,t3_l746cr,"[{'all_awardings': [], 'allow_live_comments': ...",1611857406,dannylenwinn,1.611850e+09,"{'all_awardings': [], 'allow_live_comments': F...",ForUnitedStates,t5_2i3wxn,2754,17


In [50]:
submission = reddit.submission('t3_p8ibgd')
submission.title

NotFound: received 404 HTTP response

In [40]:
r['crosspost_parent_list'][0]

IndexError: list index out of range

In [26]:
df2 = df2[df2['crosspost_parent'].notna()].reset_index()

In [52]:

df2.to_pickle(f'../../Files/test.pickle')

IndexError: list index out of range

# PySpark Test

In [6]:
from pyspark import SparkConf, SparkContext 
sc = SparkContext("local[4]", 'spark_test')





ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=spark_test, master=local[4]) created by __init__ at /tmp/ipykernel_320226/2751826978.py:2 

In [36]:
lines = sc.textFile("Files/RS_2005-07.bz2", )

In [42]:
# lines.map(lambda x: x[0]).take(1)
lines.take(1)

['{"archived":true,"author":"zaz","author_flair_background_color":null,"author_flair_css_class":null,"author_flair_richtext":[],"author_flair_text":null,"author_flair_text_color":null,"author_flair_type":"text","brand_safe":true,"can_gild":true,"contest_mode":false,"created_utc":1120181161,"distinguished":null,"domain":"cnn.com","edited":false,"gilded":0,"hidden":false,"hide_score":false,"id":"204","is_crosspostable":true,"is_reddit_media_domain":false,"is_self":false,"is_video":false,"link_flair_css_class":null,"link_flair_richtext":[],"link_flair_text":null,"link_flair_text_color":"dark","link_flair_type":"text","locked":false,"media":null,"media_embed":{},"no_follow":false,"num_comments":0,"num_crossposts":0,"over_18":false,"parent_whitelist_status":"all_ads","permalink":"\\/r\\/reddit.com\\/comments\\/204\\/cnncom_dj_admits_false_tale_about_missing_teen\\/","rte_mode":"markdown","score":4,"secure_media":null,"secure_media_embed":{},"selftext":"","send_replies":true,"spoiler":false,